# 2º Módulo de trabajo.
Vamos a trabajar los llamados Frameworks para Agentics o Agentics Frameworks.

En estos módulos lo que vamos a trabajar con agentes que generan correos electrónicos para venta fría.

1. Flujo de trabajo del agente
2. Uso de herrammientas para llamar a las funciones
3. COlaboración de los agentes mediante herramietnas y transferencias


## Cómo vamos a hacer esta tarea.
Para poder realizar esta tareaa vamos a utilizar [Sendgrid](https://sendgrid.com) que nos ayudará a enviar correos electrónicos.


In [42]:
import os
import asyncio
from typing import Dict

from dotenv import load_dotenv

from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent

import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content

In [43]:
load_dotenv(override=True)

True

## Vamos a crear primero el flujo de trabajo del agente

In [44]:
instrucciones1 = """Eres un asistente virtual que ayuda a los usuarios a encontrar información sobre analítica y ciencia de datos. Responde de manera clara y concisa y proporciona ejemplos cuando sea necesario. Cuando no sepas algo, di que no lo sabes y envíale a https://analaizer.digital o bien que mande un email a hola@javilazar.es. Eres un analísta muy técnico y te gusta explicar las cosas de manera sencilla. Te gusta ayudar a la gente a entender conceptos complejos de una manera fácil y accesible."""
instrucciones2 = """ Eres un asistente virtual para trabajadores sobre análisis de datos y ciencia de datos. Tu punto fuerte es la estrategia, donde al tener los datos que puedes conseguir a trabes de diferentes fuentes, puedes ayudar a los usuarios a tomar decisiones informadas. Responde de manera clara y concisa y proporciona ejemplos cuando sea necesario. Cuando no sepas algo, di que no lo sabes y envíale a https://analaizer.digital o bien que mande un email a hola@javilazaro.es. Además de ser un analista, eres un experto en marketing digital y te gusta ayudar a la gente a entender conceptos complejos de una manera fácil y accesible aportando insights."""
instrucciones3 = """Eres un vendedor que te dedicas sobretodo a la parte de la venta del servicio a puerta fría dando a conocer los trabajos que ofrecen. Eres capaz de explicar de manera sencilla y clara los conceptos complejos de la analítica de datos y ciencia de datos. Responde de manera clara y concisa y proporciona ejemplos cuando sea necesario. Cuando no sepas algo, di que no lo sabes y envíale a https://analaizer.digital o bien que mande un email a hola@javilazaro.es. Cómo vendendor no sueles usar palabras muy técnicas, sino que te gusta hablar de manera sencilla y directa, sin rodeos. Te gusta ayudar a la gente a entender conceptos complejos de una manera fácil y accesible. Tu objetivo es vender el servicio de analítica de datos y ciencia de datos a los usuarios, por lo que debes ser persuasivo y convincente en tus respuestas. Además, eres un experto en marketing digital y te gusta ayudar a la gente a entender conceptos complejos de una manera fácil y accesible aportando insights."""

In [45]:
analytics_agent = Agent(
    name="Agente técnico",
    instructions=instrucciones1,
    model="gpt-4o-mini",
)
estrategia_agent = Agent(
    name="Agente de estrategia",
    instructions=instrucciones2,
    model="gpt-4o-mini",
)
ventas_agent = Agent(
    name="Agente de ventas",
    instructions=instrucciones3,
    model="gpt-4o-mini",
)

In [46]:
result = Runner.run_streamed(ventas_agent, "Tienes que vender un servicio de analítica de datos y ciencia de datos a un cliente que no sabe lo que es y que lo puede necesitar. Escríbele un email de presentación del servicio. El email será escrito a un cliente frío.")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Asunto: Descunto: Descubre cómoubre cómo los los datos datos pueden impulsar tu negocio

 pueden impulsar tu negocio

Hola [NombreHola [Nombre del Cliente],

 del Cliente],

Espero que teEspero que te encuentres bien encuentres bien. Soy [. Soy [Tu Nombre]Tu Nombre] y quería tomar y quería tomar un momento para un momento para presentarte un presentarte un servicio que podría servicio que podría transformar la manera transformar la manera en la que en la que tomas decisiones tomas decisiones en tu negocio en tu negocio: la anal: la analítica de datosítica de datos y la ciencia y la ciencia de datos.

 de datos.

En un mundoEn un mundo donde la información donde la información está por todas está por todas partes, saber partes, saber aprovecharla puede aprovecharla puede ser la clave ser la clave para el éxito para el éxito. Pero,. Pero, ¿qué significa ¿qué significa realmente la anal realmente la analítica de datosítica de datos? En términos? En términos simples, se simples, se trata d

In [47]:
message = "Escribe un email de presentación del servicio de analítica de datos y ciencia de datos a un cliente que no sabe lo que es y que lo puede necesitar. El email será escrito a un cliente frío."

with trace("Correo electrónico de presentación del servicio"):
    results = await asyncio.gather(
        Runner.run(analytics_agent, message),
        Runner.run(estrategia_agent, message),
        Runner.run(ventas_agent, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output)
    print("\n" + "=" * 80 + "\n")

Asunto: Impulsa tu negocio con la Analítica y Ciencia de Datos

Hola [Nombre del Cliente],

Espero que te encuentres bien. Mi nombre es [Tu Nombre] y soy [tu puesto] en [Nombre de tu Empresa]. Quiero presentarte cómo la analítica de datos y la ciencia de datos pueden transformar tu negocio y ayudarte a tomar decisiones más informadas.

La analítica de datos consiste en el proceso de recopilar, procesar y analizar datos para extraer información útil. Por otro lado, la ciencia de datos combina herramientas estadísticas y técnicas de programación para crear modelos que permiten predecir tendencias y comportamientos futuros.

**¿Cómo puede beneficiarte esto?**

1. **Toma de Decisiones Basadas en Datos**: En lugar de basarte en suposiciones, podrás fundamentar tus decisiones en datos concretos.
   
2. **Identificación de Oportunidades**: Al analizar tus datos, podemos descubrir áreas de mejora y nuevas oportunidades de negocio.

3. **Optimización de Procesos**: Analizar tu operativa te perm

In [48]:
sales_picker = Agent(
    name="Sales Picker",
    instructions="Elige el correo electrónico de ventas más adecuado para enviar al cliente. \
        Elige el correo electrónico que mejor se adapte a las necesidades del cliente y que sea más persuasivo.",
    model="gpt-4o-mini",
)

In [49]:
message = "Escribe un email de presentación del servicio de analítica de datos y ciencia de datos a un cliente que no sabe lo que es y que lo puede necesitar. El email será escrito a un cliente frío. Intenta vender el servicio de manera persuasiva y convincente según tus fortalezas."

with trace("Selección del correo electrónico de ventas"):
    results = await asyncio.gather(
        Runner.run(analytics_agent, message),
        Runner.run(estrategia_agent, message),
        Runner.run(ventas_agent, message),
    )

    outputs = [result.final_output for result in results]

    emails = "meails de ventas:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"El mejor correo electrónico de ventas es:\n{best.final_output}\n")



El mejor correo electrónico de ventas es:
Elegir el correo electrónico más adecuado depende del enfoque que quieras darle y del tono que creas que resuena mejor con el cliente. Aquí tienes un análisis breve de cada uno:

1. **Asunto: Potencia tu Negocio con Analítica de Datos**
   - **Pros**: Ofrece un enfoque directo y profesional, destacando los beneficios de inmediato. Además, proporciona un resumen claro de lo que se puede obtener a través de la analítica.
   - **Contras**: Puede parecer un poco técnico para un cliente que no esté familiarizado con la analítica de datos.

2. **Asunto: Impulsa tu Negocio con Análisis de Datos**
   - **Pros**: Se presenta de manera más amigable y accesible, promete beneficios tangibles y utiliza un ejemplo práctico que puede captar la atención del cliente.
   - **Contras**: Puede carecer de profundidad técnica en comparación con el primero.

3. **Asunto: Potencia tu negocio con nuestros servicios de Analítica de Datos**
   - **Pros**: Aborda la neces

Revisión de las peticiones: https://platform.openai.com/traces

## Uso de herramietnas
Ahora vmoas a añadir las herramientas con las que vamos a trabajar.
Este códgo será un JSON repetitivo y la funcion `handle_tool_calls()` con las lógica `if`

In [50]:
analytics_agent = Agent (
    name= "analytics_agent",
    instructions=instrucciones1,
    model="gpt-4o-mini"
    )

estrategia_agent = Agent (
    name= "estrategia_agent",
    instructions=instrucciones2,
    model="gpt-4o-mini" 
    )

ventas_agent = Agent (
    name= "ventas_agent",
    instructions=instrucciones3,
    model="gpt-4o-mini"
)

In [51]:
analytics_agent

Agent(name='analytics_agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='Eres un asistente virtual que ayuda a los usuarios a encontrar información sobre analítica y ciencia de datos. Responde de manera clara y concisa y proporciona ejemplos cuando sea necesario. Cuando no sepas algo, di que no lo sabes y envíale a https://analaizer.digital o bien que mande un email a hola@javilazar.es. Eres un analísta muy técnico y te gusta explicar las cosas de manera sencilla. Te gusta ayudar a la gente a entender conceptos complejos de una manera fácil y accesible.', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guar

## Interacción de agentes con herramietnas
Ahora es el momento de que los agentes que hemos ido creando interactuen con las herramientas que vamos a configurar.
Es el momento de sacar la parte que hemos aprendido de JSON a relucir usando decoradores en esta caso 
```python
 @function_tool
 ```

In [52]:
@function_tool
def send_email(body: str):
    """Envia un correo electrónico utilizando SendGrid.

    Args:
        body (str): El cuerpo del correo electrónico a enviar.
    """

    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("pichu_2707@hotmail.com")
    to_email = To("pichu2707@gmail.com")
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Correo de información de analítica digital", content).get()
    return {"status": "success"}

In [53]:
send_email

FunctionTool(name='send_email', description='Envia un correo electrónico utilizando SendGrid.', params_json_schema={'properties': {'body': {'description': 'El cuerpo del correo electrónico a enviar.', 'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B0DD745080>, strict_json_schema=True, is_enabled=True)

## Conversor de Agente en herramienta
Ahora además podemos convertir un Agente en una herramienta

In [54]:
tool1  = ventas_agent.as_tool(tool_name="sales_agent", tool_description="Herramienta para enviar correos electrónicos de ventas")

## Es el momento de reunir todas las herramientas:
Una herramienta para cada uno de nuestro s agentes de analítica para redactar los correos electrónicos.
Y una herramietna para nuiestra función de envío de correos electrónicos.


In [55]:
description = "Escribe un correo eletrónico de ventas para vender un servicio de analítica de datos y ciencia de datos a un cliente que no sabe lo que es y que lo puede necesitar. El email será escrito a un cliente frío."

tool3 = analytics_agent.as_tool(tool_name = "sales_agent3", tool_description=description)
    
    
tool2 = estrategia_agent.as_tool(tool_name = "sales_agent2", tool_description=description)
    

tool1 = ventas_agent.as_tool(tool_name = "sales_agent1", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools
    

[FunctionTool(name='sales_agent3', description='Escribe un correo eletrónico de ventas para vender un servicio de analítica de datos y ciencia de datos a un cliente que no sabe lo que es y que lo puede necesitar. El email será escrito a un cliente frío.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent3_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B0DDB19940>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Escribe un correo eletrónico de ventas para vender un servicio de analítica de datos y ciencia de datos a un cliente que no sabe lo que es y que lo puede necesitar. El email será escrito a un cliente frío.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'typ

## Vamos a crear un encargado de las ventas:
Es el momento de crear al project manager de ventas para la planificación de los correos.

In [56]:
instruccions = """Eres un gerente de ventas y tienes que enviar un correo electrónico de ventas a un cliente que no sabe lo que es la analítica de datos y ciencia de datos. Tu objetivo es vender el servicio de analítica de datos y ciencia de datos a los usuarios, por lo que debes ser persuasivo y convincente en tus respuestas. Además, eres un experto en marketing digital y te gusta ayudar a la gente a entender conceptos complejos de una manera fácil y accesible aportando insights. No creas los correos electrónicos, sino que los envías directamente a través de la herramienta de envío de correos electrónicos. Tu objetivo es vender el servicio de analítica de datos y ciencia de datos a los usuarios, por lo que debes ser persuasivo y convincente en tus respuestas. Además, eres un experto en marketing digital y te gusta ayudar a la gente a entender conceptos complejos de una manera fácil y accesible aportando insights. Elige el correo eletronico y usa la herramienta de sendmail para enviar el mejor correo electrónico (y solo el mejor) al usuario."""

sales_manager = Agent(name="Project Manager de ventas", instructions=instruccions, model="gpt-4o-mini", tools=tools)

message = " Envía el correo electrónico de ventas frío dirigido a el ejecutivo de la empresa Analaizer Digital"

with trace("Project Manager de ventas"):
    result = await Runner.run(sales_manager, message)

## Si no recibes el correo electrónico:
Si no recibes el correo electrónico deberísa revisar los siguientes puntos:
* Revisa que no esté en tu carpeta de **correo no deseado** o **spam**
* Imprime el resultado y así comprueba si recibes **errores sobre los SSL**, si es así tienes más información en [este sitio](https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d)
* Tambíen revisa el **seguimiento de OpenAI** y revisa la configuración de Sendgrid para poder encontrar pistas a tu problema.



En el caso de recibir errores de certificado SSL, ejecuta en tu terminal el siguiente código:

```python
import certifi
import os
os.environ['SSL_CERT_FILE']=certifi.where
```

## Los Handoff
Las transferencias o Handoffs reprensenta una forma en que un agente puede delegar en otro agente, transfiriéndole el control.
 Las transeferencias y los agentes como herramietnas son similares:

* En ambos casos, un agente puede colaborar con otro.
* Con las herramietnas, el control se transfiere.
* Con las transferencias el control se transfiere.

Un agente puede tener una gran cantidad de herramientas o puede tener handoffs, en este caso los handoffs son los agentes en los que delega el trabajo. 
Podríamos decir que el Handoff es delegar el trabajo a otro agente más espcilizado, delegando el trabajo por completo.


In [57]:
subejct_instructions = """Eres un gerente de ventas y tienes que enviar un correo electrónico de ventas a un cliente que no sabe lo que es la analítica de datos y ciencia de datos. Tu objetivo es vender el servicio de analítica de datos y ciencia de datos a los usuarios, por lo que debes ser persuasivo y convincente en tus respuestas. Además, eres un experto en marketing digital y te gusta ayudar a la gente a entender conceptos complejos de una manera fácil y accesible aportando insights. No creas los correos electrónicos, sino que los envías directamente a través de la herramienta de envío de correos electrónicos. Tu objetivo es vender el servicio de analítica de datos y ciencia de datos a los usuarios, por lo que debes ser persuasivo y convincente en tus respuestas. Además, eres un experto en marketing digital y te gusta ayudar a la gente a entender conceptos complejos de una manera fácil y accesible aportando insights. Elige el correo eletronico y usa la herramienta de sendmail para enviar el mejor correo electrónico (y solo el mejor) al usuario."""

html_instructions = """Puedes convertir un cuerpo  del correo electrónico de texto a HTML. Se te proporcionará el cuerpo del correo electrónico en texto plano y deberás convertirlo a HTML para enviarlo. Asegúrate de que el formato sea adecuado para un correo electrónico profesional y que incluya todos los elementos necesarios, como encabezados, párrafos y enlaces. Tu objetivo es hacer que el correo electrónico sea visualmente atractivo y fácil de leer para el destinatario. Además, asegúrate de que el HTML sea compatible con la mayoría de los clientes de correo electrónico."""

subject_writer = Agent(
                    name="Escrito de asuntos de correo electrónico",
                    instructions=subejct_instructions,
                    model="gpt-4o-mini"
                    )

# Lo convertimos en una herramienta
subject_tool = subject_writer.as_tool(
                                    tool_name="subject_writer",
                                    tool_description="Convierte un cuerpo de correo electrónico de texto a un asunto adecuado para el correo electrónico.",
                                    )   

html_converter = Agent(
                    name="Conversor de cuerpo de correo electrónico a HTML",
                    instructions=html_instructions,
                    model="gpt-4o-mini",
                    )

# Lo convertimos en una herramienta
html_tool = html_converter.as_tool(
                                    tool_name="HTML_converter",
                                    tool_description="Convierte un cuerpo de correo electrónico de texto a un cuerpo de correo electrónico HTML.",
                                    )



In [58]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """Envia un correo electrónico en formato HTML utilizando SendGrid.

    Args:
        subject (str): El asunto del correo electrónico.
        body (str): El cuerpo del correo electrónico en formato HTML.
    
    Returns:
        Dict[str, str]: Un diccionario con el estado del envío del correo electrónico.
    """
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("pichu_2707@hotmail.com")
    to_email = To("pichu2707@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {
        "status": "success"
    }

tools = [
    subject_tool,
    html_tool,
    send_html_email
    ]

tools

[FunctionTool(name='subject_writer', description='Convierte un cuerpo de correo electrónico de texto a un asunto adecuado para el correo electrónico.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B0DD9E7EC0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='HTML_converter', description='Convierte un cuerpo de correo electrónico de texto a un cuerpo de correo electrónico HTML.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'HTML_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B0DDB70400>, strict_json_schema=True, is_enabled=True),
 Func

In [59]:
instructions = "Eres un formateador y remitente de correos electróinicos. Recibes el cuerpo de un correo electrónico para enviarlo. Primero usa la herramienta de subject_writer para generar un asunto adecuado para el correo electrónico, despúes usa la herramienta de html_convierter oara convertir el cuerpo del correo eletrónico a HTML y finalmente usa la herramienta de send_html_email para enviar el correo electrónico. Asegúrate de que el asunto y el cuerpo del correo electrónico sean adecuados para un correo electrónico profesional y que incluyan todos los elementos necesarios, como encabezados, párrafos y enlaces. Tu objetivo es hacer que el correo electrónico sea visualmente atractivo y fácil de leer para el destinatario."

emailer_agent = Agent(
    name="Email Agent",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convierte un email a HTML y lo envia al destinatario."
)

Ahora tenemos tres herramientas y una transferencia

In [60]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent3', description='Escribe un correo eletrónico de ventas para vender un servicio de analítica de datos y ciencia de datos a un cliente que no sabe lo que es y que lo puede necesitar. El email será escrito a un cliente frío.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent3_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B0DDB19940>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Escribe un correo eletrónico de ventas para vender un servicio de analítica de datos y ciencia de datos a un cliente que no sabe lo que es y que lo puede necesitar. El email será escrito a un cliente frío.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type

In [61]:
sales_manager_instructions = """Eres el gerente de ventas que trabaja en Analaizer.Digital. Utilizas las herramientas que se te han proporcionado para generar correos eletrónicos de venta en frío. Nunca generas lo correos de ventas tu mismo, siempre utilizas las herramientas. Prueba las tres herramientas del agente de ventas al menos una vez antes de elegir la mejor. Puedes usar las herramietnas múltiples veces si no estás satisfecho con los resultados del primer intento. Selecciona el mejor correo electrónico de ventas según cual crees qeu será más efectivo a la hora de convertir a un cliente frío en un cliente potencial. Utiliza la herramienta de send_html_email para enviar el correo electrónico al cliente. Asegúrate de que el asunto y el cuerpo del correo electrónico sean adecuados para un correo electrónico profesional y que incluyan todos los elementos necesarios, como encabezados, párrafos y enlaces. Tu objetivo es hacer que el correo electrónico sea visualmente atractivo y fácil de leer para el destinatario. Despúes de elegir el correo electrónico, tranfieres al agente Email Manager para formatear y enviar el correo eletrónico."""

sales_manager = Agent(
    name="Project Manager de ventas",
    instructions=sales_manager_instructions,
    model="gpt-4o-mini",
    tools=tools,
    handoffs=handoffs,
)

message = "Envia un correo de ventas en frío a un cliente que no sabe lo que es la analítica de datos y ciencia de datos. El email será escrito a un cliente frío. Intenta vender el servicio de manera persuasiva y convincente según tus fortalezas."

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)
